# 데이터 합성.

* 코비스 데이터 및 영화진흥위원회 데이터 합치기
* 네이버 api 및 TMDB api를 활용하여 필요한 데이터 수집.

## 🔹 깃헙 레포지토리 가져오기 및 데이터 불러오기.

깃 레포지토리를 통째로 클론해서 작업 진행.

In [1]:
# 깃헙 레포지토리 전체를 코랩 환경에 복사하기.
!git clone https://github.com/zzuupp/-Screen-Wizard.git

Cloning into '-Screen-Wizard'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 39 (delta 7), reused 37 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (39/39), 4.23 MiB | 12.37 MiB/s, done.
Resolving deltas: 100% (7/7), done.


현재 위치 확인하기.

In [2]:
!pwd

/content


git 으로 클론한 위치로 이동해야함, clone 이후 이동이 없어 현 위치가 `/content`인 상태.
* `clone`은 현재 깃헙 레포지토리의 환경을 복사한 것임(복사한 주소로 이동한 상태가 아님)
* 먼저 Git 저장소(레포지토리)가  잘 복사되어 있는지 확인하기.

In [3]:
!ls -a

.  ..  .config	sample_data  -Screen-Wizard


`Repository : -Screen-Wizard` 로 이동

In [4]:
# -S 가 옵션으로 인식되어 ./ 해야함
%cd ./-Screen-Wizard

/content/-Screen-Wizard


최신 코드 가져오기.
* 항상 작업 전 필수로 진행하기.

In [5]:
!git pull origin main #최신 코드 가져오기.

From https://github.com/zzuupp/-Screen-Wizard
 * branch            main       -> FETCH_HEAD
Already up to date.


브랜치 목록 업데이트.
* 원격 저장소에 존재하는 모든 브랜치 정보 가져와서 업데이트하기.

In [6]:
!git fetch origin # 원격 브랜치 목록 업데이트

원격 브랜치 목록 확인.

In [7]:
! git branch -r

  origin/HEAD -> origin/main
  origin/data/data-cleansing
  origin/main


브랜치 조회 : 로컬 및 원격의 모든 브랜치 조회하기.
* `*`는 현재 체크아웃(머물러있는)된 브랜치이다.
* ➡ `remotes/origin/브랜치명`
→ 원격(GitHub)에 있는 브랜치를 의미!

In [8]:
!git branch -a

* main
  remotes/origin/HEAD -> origin/main
  remotes/origin/data/data-cleansing
  remotes/origin/main


현재 브랜치는 안보이는 상태이다(main 값만 보여짐)

In [9]:
!git branch

* main


왜냐면 원격 브랜치에는 존재하고, 현재 내 로컬 브랜치에는 가져오지 않았기 때문.
* 참고 : ✅ [원격 브랜치 vs. 로컬 브랜치](https://www.notion.so/vs-190bdc05b2b1804686fec296d989dea5?pvs=4)

❗️ 드디어...! 대망의 브랜치 이동!!!

In [10]:
!git switch --track origin/data/data-cleansing

Branch 'data/data-cleansing' set up to track remote branch 'data/data-cleansing' from 'origin'.
Switched to a new branch 'data/data-cleansing'


브랜치 연결(이동) 후 확인.

In [11]:
!git branch -a


* data/data-cleansing
  main
  remotes/origin/HEAD -> origin/main
  remotes/origin/data/data-cleansing
  remotes/origin/main


이후 현재 브랜치의 최신 코드를 가져오기. </br>
❗️ 코드를 가져올 때 main이 아니라, 현재 있는 브랜치에서 pull을 해야한다. </br>



In [12]:
!git pull  # 현재 브랜치(data/data-cleansing)의 최신 코드 가져오기
!git merge origin/main  # main 브랜치의 변경 사항을 병합하기

Already up to date.
Updating d7ced5c..e791521
Fast-forward
 Data/raw/Kobis_data/kobis_data_excel.xlsx                      | Bin 0 -> 2659521 bytes
 Data/{ => raw}/after/.DS_Store                                 | Bin
 Data/{ => raw}/after/after_actors.csv                          |   0
 Data/{ => raw}/after/after_boxoffice.csv                       |   0
 Data/{ => raw}/after/after_companies_df.csv                    |   0
 Data/{ => raw}/after/after_directors.csv                       |   0
 Data/{ => raw}/after/after_genres.csv                          |   0
 Data/{ => raw}/after/after_movie.csv                           |   0
 Data/{ => raw}/after/after_showtypes.csv                       |   0
 Data/{ => raw}/before/.DS_Store                                | Bin
 Data/{ => raw}/before/before_actors.csv                        |   0
 Data/{ => raw}/before/before_boxoffice.csv                     |   0
 Data/{ => raw}/before/before_companies.csv                     |   0
 Data/{ => r

---

In [13]:
!pwd # 경로 다시 확인

/content/-Screen-Wizard


레포지토 내부의 데이터 확인.

In [14]:
!ls -R /content/-Screen-Wizard/Data/raw/

/content/-Screen-Wizard/Data/raw/:
after  before  Kobis_data

/content/-Screen-Wizard/Data/raw/after:
after_actors.csv     after_companies_df.csv  after_genres.csv  after_showtypes.csv
after_boxoffice.csv  after_directors.csv     after_movie.csv

/content/-Screen-Wizard/Data/raw/before:
before_actors.csv     before_companies.csv  before_genres.csv  before_showtypes.csv
before_boxoffice.csv  before_directors.csv  before_movie.csv

/content/-Screen-Wizard/Data/raw/Kobis_data:
kobis_data_excel.xlsx


## １. 데이터 가져오기(필요한 라이브러리 포함)

In [15]:
import pandas as pd
import numpy as np

In [16]:
# 데이터 가져오기.
kobis = '/content/-Screen-Wizard/Data/raw/Kobis_data/kobis_data_excel.xlsx'
after_box_office = '/content/-Screen-Wizard/Data/raw/after/after_boxoffice.csv'
before_box_office = '/content/-Screen-Wizard/Data/raw/before/before_boxoffice.csv'

In [17]:
kobis = pd.read_excel(kobis)
after_boxoffice = pd.read_csv(after_box_office)
before_boxoffice = pd.read_csv(before_box_office)

### 𝟣-𝟣 Kobis 데이터 전처리.

In [18]:
kobis

,■ 개봉일람_연도별,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,- 조회일: 2025-01-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,- 출처: 영화진흥위원회 통합전산망 (http://www.kobis.or.kr),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,▶ 검색조건 [ 조회기간 : 2010~2024 국적 : 전체 영화명 : 전체...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,<,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,순번,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,전국,전국,전국,서울,서울,장르,등급,영화구분,영화코드
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20381,20376,av감독의 아내,쿠도 마사노리,NaN,(주)도키엔터테인먼트,(주)도키엔터테인먼트,2016-12-22 00:00:00,개봉영화,장편,일본,1,8000,1,0,0,성인물(에로),청소년관람불가,일반영화,e
20382,20377,mz 자매의 목적,함성한,P&미디어,NaN,P&미디어,2023-08-15 00:00:00,개봉영화,장편,한국,1,1000,1,0,0,NaN,청소년관람불가,일반영화,e
20383,20378,만화방,허재형,(주)영진크리에이티브,NaN,(주)영진크리에이티브,2012-02-29 00:00:00,개봉영화,장편,한국,0,0,0,0,0,성인물(에로),청소년관람불가,일반영화,e
20384,20379,클래쉬,레 탄 선,NaN,(주)소나무픽쳐스,(주)소나무픽쳐스,2011-08-11 00:00:00,개봉영화,장편,베트남,0,0,0,0,0,액션,청소년관람불가,일반영화,e


불필요한 인덱스 삭제.

In [19]:
kobis = kobis.drop(kobis.index[0:4])
kobis

,■ 개봉일람_연도별,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
4,순번,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,전국,전국,전국,서울,서울,장르,등급,영화구분,영화코드
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,스크린수,매출액,관객수,매출액,관객수,NaN,NaN,NaN,NaN
6,1,명량,김한민,(주)빅스톤픽쳐스,NaN,(주)씨제이이엔엠,2014-07-30 00:00:00,개봉영화,장편,한국,1587,135748398910,17613682,33121225810,4163666,사극,15세이상관람가,일반영화,20129370
7,2,극한직업,이병헌,"(주)어바웃잇,영화사 해그림 주식회사,(주)씨제이이엔엠",NaN,(주)씨제이이엔엠,2019-01-23 00:00:00,개봉영화,장편,한국,1978,139647979516,16264944,31858660536,3638287,코미디,15세이상관람가,일반영화,20182530
8,3,신과함께-죄와 벌,김용화,"리얼라이즈픽쳐스(주),(주)덱스터스튜디오",NaN,롯데쇼핑㈜롯데엔터테인먼트,2017-12-20 00:00:00,개봉영화,장편,한국,1912,115698654137,14410754,27530825087,3346172,판타지,12세이상관람가,일반영화,20150976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20381,20376,av감독의 아내,쿠도 마사노리,NaN,(주)도키엔터테인먼트,(주)도키엔터테인먼트,2016-12-22 00:00:00,개봉영화,장편,일본,1,8000,1,0,0,성인물(에로),청소년관람불가,일반영화,e
20382,20377,mz 자매의 목적,함성한,P&미디어,NaN,P&미디어,2023-08-15 00:00:00,개봉영화,장편,한국,1,1000,1,0,0,NaN,청소년관람불가,일반영화,e
20383,20378,만화방,허재형,(주)영진크리에이티브,NaN,(주)영진크리에이티브,2012-02-29 00:00:00,개봉영화,장편,한국,0,0,0,0,0,성인물(에로),청소년관람불가,일반영화,e
20384,20379,클래쉬,레 탄 선,NaN,(주)소나무픽쳐스,(주)소나무픽쳐스,2011-08-11 00:00:00,개봉영화,장편,베트남,0,0,0,0,0,액션,청소년관람불가,일반영화,e


4번 5번행을 합친 후 컬럼 지정.

In [20]:
# 첫 번째 & 두 번째 행을 결합하여 컬럼명으로 사용
kobis.columns = kobis.iloc[0].fillna('') +'_'+ kobis.iloc[1].fillna('')

# 좌우에 존재하는 '_' 제거(결합하는 과정에서 NaN을 결합한다면 _만 붙여지기 때문)
kobis.columns = [col.strip('_') for col in kobis.columns]

# 필요없는 행(2개) 제거.
kobis = kobis[2:].reset_index(drop = True)
kobis

,순번,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,전국_스크린수,전국_매출액,전국_관객수,서울_매출액,서울_관객수,장르,등급,영화구분,영화코드
0,1,명량,김한민,(주)빅스톤픽쳐스,NaN,(주)씨제이이엔엠,2014-07-30 00:00:00,개봉영화,장편,한국,1587,135748398910,17613682,33121225810,4163666,사극,15세이상관람가,일반영화,20129370
1,2,극한직업,이병헌,"(주)어바웃잇,영화사 해그림 주식회사,(주)씨제이이엔엠",NaN,(주)씨제이이엔엠,2019-01-23 00:00:00,개봉영화,장편,한국,1978,139647979516,16264944,31858660536,3638287,코미디,15세이상관람가,일반영화,20182530
2,3,신과함께-죄와 벌,김용화,"리얼라이즈픽쳐스(주),(주)덱스터스튜디오",NaN,롯데쇼핑㈜롯데엔터테인먼트,2017-12-20 00:00:00,개봉영화,장편,한국,1912,115698654137,14410754,27530825087,3346172,판타지,12세이상관람가,일반영화,20150976
3,4,국제시장,윤제균,"(주)제이케이필름,(주)씨제이이엔엠",NaN,(주)씨제이이엔엠,2014-12-17 00:00:00,개봉영화,장편,한국,966,110828014630,14245998,25842519330,3233946,드라마,12세이상관람가,일반영화,20137048
4,5,어벤져스: 엔드게임,"안소니 루소,조 루소",NaN,월트디즈니컴퍼니코리아 유한책임회사,월트디즈니컴퍼니코리아 유한책임회사,2019-04-24 00:00:00,개봉영화,장편,미국,2835,122182694160,13934592,33577136860,3597963,액션,12세이상관람가,일반영화,20184889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20375,20376,av감독의 아내,쿠도 마사노리,NaN,(주)도키엔터테인먼트,(주)도키엔터테인먼트,2016-12-22 00:00:00,개봉영화,장편,일본,1,8000,1,0,0,성인물(에로),청소년관람불가,일반영화,e
20376,20377,mz 자매의 목적,함성한,P&미디어,NaN,P&미디어,2023-08-15 00:00:00,개봉영화,장편,한국,1,1000,1,0,0,NaN,청소년관람불가,일반영화,e
20377,20378,만화방,허재형,(주)영진크리에이티브,NaN,(주)영진크리에이티브,2012-02-29 00:00:00,개봉영화,장편,한국,0,0,0,0,0,성인물(에로),청소년관람불가,일반영화,e
20378,20379,클래쉬,레 탄 선,NaN,(주)소나무픽쳐스,(주)소나무픽쳐스,2011-08-11 00:00:00,개봉영화,장편,베트남,0,0,0,0,0,액션,청소년관람불가,일반영화,e


In [21]:
kobis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20380 entries, 0 to 20379
Data columns (total 19 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   순번       20380 non-null  object
 1   영화명      20380 non-null  object
 2   감독       18127 non-null  object
 3   제작사      7067 non-null   object
 4   수입사      10905 non-null  object
 5   배급사      20357 non-null  object
 6   개봉일      20380 non-null  object
 7   영화유형     20380 non-null  object
 8   영화형태     20379 non-null  object
 9   국적       20380 non-null  object
 10  전국_스크린수  20380 non-null  object
 11  전국_매출액   20380 non-null  object
 12  전국_관객수   20380 non-null  object
 13  서울_매출액   20373 non-null  object
 14  서울_관객수   20380 non-null  object
 15  장르       20249 non-null  object
 16  등급       20379 non-null  object
 17  영화구분     20380 non-null  object
 18  영화코드     20380 non-null  object
dtypes: object(19)
memory usage: 3.0+ MB


결측값 여부 확인 : `개봉일`

In [23]:
dataime_nan = kobis[kobis['개봉일'].isna()]

In [22]:
kobis['개봉일'] = pd.to_datetime(kobis['개봉일'])
kobis = kobis.drop('순번', axis = 1)

In [24]:
kobis

,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,전국_스크린수,전국_매출액,전국_관객수,서울_매출액,서울_관객수,장르,등급,영화구분,영화코드
0,명량,김한민,(주)빅스톤픽쳐스,NaN,(주)씨제이이엔엠,2014-07-30,개봉영화,장편,한국,1587,135748398910,17613682,33121225810,4163666,사극,15세이상관람가,일반영화,20129370
1,극한직업,이병헌,"(주)어바웃잇,영화사 해그림 주식회사,(주)씨제이이엔엠",NaN,(주)씨제이이엔엠,2019-01-23,개봉영화,장편,한국,1978,139647979516,16264944,31858660536,3638287,코미디,15세이상관람가,일반영화,20182530
2,신과함께-죄와 벌,김용화,"리얼라이즈픽쳐스(주),(주)덱스터스튜디오",NaN,롯데쇼핑㈜롯데엔터테인먼트,2017-12-20,개봉영화,장편,한국,1912,115698654137,14410754,27530825087,3346172,판타지,12세이상관람가,일반영화,20150976
3,국제시장,윤제균,"(주)제이케이필름,(주)씨제이이엔엠",NaN,(주)씨제이이엔엠,2014-12-17,개봉영화,장편,한국,966,110828014630,14245998,25842519330,3233946,드라마,12세이상관람가,일반영화,20137048
4,어벤져스: 엔드게임,"안소니 루소,조 루소",NaN,월트디즈니컴퍼니코리아 유한책임회사,월트디즈니컴퍼니코리아 유한책임회사,2019-04-24,개봉영화,장편,미국,2835,122182694160,13934592,33577136860,3597963,액션,12세이상관람가,일반영화,20184889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20375,av감독의 아내,쿠도 마사노리,NaN,(주)도키엔터테인먼트,(주)도키엔터테인먼트,2016-12-22,개봉영화,장편,일본,1,8000,1,0,0,성인물(에로),청소년관람불가,일반영화,e
20376,mz 자매의 목적,함성한,P&미디어,NaN,P&미디어,2023-08-15,개봉영화,장편,한국,1,1000,1,0,0,NaN,청소년관람불가,일반영화,e
20377,만화방,허재형,(주)영진크리에이티브,NaN,(주)영진크리에이티브,2012-02-29,개봉영화,장편,한국,0,0,0,0,0,성인물(에로),청소년관람불가,일반영화,e
20378,클래쉬,레 탄 선,NaN,(주)소나무픽쳐스,(주)소나무픽쳐스,2011-08-11,개봉영화,장편,베트남,0,0,0,0,0,액션,청소년관람불가,일반영화,e


2011년 이전 데이터 삭제.

In [25]:
filtered_kobis = kobis[kobis['개봉일'] >= '2011-01-01']
filtered_kobis.rename(columns = {'장르':'대표장르'})

,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,전국_스크린수,전국_매출액,전국_관객수,서울_매출액,서울_관객수,대표장르,등급,영화구분,영화코드
0,명량,김한민,(주)빅스톤픽쳐스,NaN,(주)씨제이이엔엠,2014-07-30,개봉영화,장편,한국,1587,135748398910,17613682,33121225810,4163666,사극,15세이상관람가,일반영화,20129370
1,극한직업,이병헌,"(주)어바웃잇,영화사 해그림 주식회사,(주)씨제이이엔엠",NaN,(주)씨제이이엔엠,2019-01-23,개봉영화,장편,한국,1978,139647979516,16264944,31858660536,3638287,코미디,15세이상관람가,일반영화,20182530
2,신과함께-죄와 벌,김용화,"리얼라이즈픽쳐스(주),(주)덱스터스튜디오",NaN,롯데쇼핑㈜롯데엔터테인먼트,2017-12-20,개봉영화,장편,한국,1912,115698654137,14410754,27530825087,3346172,판타지,12세이상관람가,일반영화,20150976
3,국제시장,윤제균,"(주)제이케이필름,(주)씨제이이엔엠",NaN,(주)씨제이이엔엠,2014-12-17,개봉영화,장편,한국,966,110828014630,14245998,25842519330,3233946,드라마,12세이상관람가,일반영화,20137048
4,어벤져스: 엔드게임,"안소니 루소,조 루소",NaN,월트디즈니컴퍼니코리아 유한책임회사,월트디즈니컴퍼니코리아 유한책임회사,2019-04-24,개봉영화,장편,미국,2835,122182694160,13934592,33577136860,3597963,액션,12세이상관람가,일반영화,20184889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20375,av감독의 아내,쿠도 마사노리,NaN,(주)도키엔터테인먼트,(주)도키엔터테인먼트,2016-12-22,개봉영화,장편,일본,1,8000,1,0,0,성인물(에로),청소년관람불가,일반영화,e
20376,mz 자매의 목적,함성한,P&미디어,NaN,P&미디어,2023-08-15,개봉영화,장편,한국,1,1000,1,0,0,NaN,청소년관람불가,일반영화,e
20377,만화방,허재형,(주)영진크리에이티브,NaN,(주)영진크리에이티브,2012-02-29,개봉영화,장편,한국,0,0,0,0,0,성인물(에로),청소년관람불가,일반영화,e
20378,클래쉬,레 탄 선,NaN,(주)소나무픽쳐스,(주)소나무픽쳐스,2011-08-11,개봉영화,장편,베트남,0,0,0,0,0,액션,청소년관람불가,일반영화,e


In [26]:
filtered_kobis.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19897 entries, 0 to 20379
Data columns (total 18 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   영화명      19897 non-null  object        
 1   감독       17644 non-null  object        
 2   제작사      6640 non-null   object        
 3   수입사      10594 non-null  object        
 4   배급사      19879 non-null  object        
 5   개봉일      19897 non-null  datetime64[ns]
 6   영화유형     19897 non-null  object        
 7   영화형태     19896 non-null  object        
 8   국적       19897 non-null  object        
 9   전국_스크린수  19897 non-null  object        
 10  전국_매출액   19897 non-null  object        
 11  전국_관객수   19897 non-null  object        
 12  서울_매출액   19891 non-null  object        
 13  서울_관객수   19897 non-null  object        
 14  장르       19766 non-null  object        
 15  등급       19896 non-null  object        
 16  영화구분     19897 non-null  object        
 17  영화코드     19897 non-null  object     

매출액 관련 컬럼 숫자화
* `전국_매출액`, `서울_매출액`

In [27]:
filtered_kobis['전국_매출액'] = filtered_kobis['전국_매출액'].astype(int)

<ipython-input-27-e7e822838b4a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_kobis['전국_매출액'] = filtered_kobis['전국_매출액'].astype(int)


In [28]:
filtered_kobis.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19897 entries, 0 to 20379
Data columns (total 18 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   영화명      19897 non-null  object        
 1   감독       17644 non-null  object        
 2   제작사      6640 non-null   object        
 3   수입사      10594 non-null  object        
 4   배급사      19879 non-null  object        
 5   개봉일      19897 non-null  datetime64[ns]
 6   영화유형     19897 non-null  object        
 7   영화형태     19896 non-null  object        
 8   국적       19897 non-null  object        
 9   전국_스크린수  19897 non-null  object        
 10  전국_매출액   19897 non-null  int64         
 11  전국_관객수   19897 non-null  object        
 12  서울_매출액   19891 non-null  object        
 13  서울_관객수   19897 non-null  object        
 14  장르       19766 non-null  object        
 15  등급       19896 non-null  object        
 16  영화구분     19897 non-null  object        
 17  영화코드     19897 non-null  object     

In [31]:
# null값 확인
filtered_kobis[filtered_kobis['서울_매출액'].isnull()]

# null값 채우기.
filtered_kobis.loc[:, '서울_매출액'] = filtered_kobis['서울_매출액'].fillna(0).infer_objects(copy=False)

In [32]:
filtered_kobis.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19897 entries, 0 to 20379
Data columns (total 18 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   영화명      19897 non-null  object        
 1   감독       17644 non-null  object        
 2   제작사      6640 non-null   object        
 3   수입사      10594 non-null  object        
 4   배급사      19879 non-null  object        
 5   개봉일      19897 non-null  datetime64[ns]
 6   영화유형     19897 non-null  object        
 7   영화형태     19896 non-null  object        
 8   국적       19897 non-null  object        
 9   전국_스크린수  19897 non-null  object        
 10  전국_매출액   19897 non-null  int64         
 11  전국_관객수   19897 non-null  object        
 12  서울_매출액   19897 non-null  int64         
 13  서울_관객수   19897 non-null  object        
 14  장르       19766 non-null  object        
 15  등급       19896 non-null  object        
 16  영화구분     19897 non-null  object        
 17  영화코드     19897 non-null  object     

#### Kobis 데이터 전처리 작업 내용.
* 1. 개봉일 dtype 변경(datetime)
* 2. 특정 컬럼명 변경  장르 -> 대표 장르
* 3. 2011년 이전 데이터 제외[(2011년 이후 데이터가 제대로 수집됨)](https://www.kobis.or.kr/kobis/business/stat/offc/searchOfficHitTotList.do?searchMode=year)
* 4. 매출관련 데이터 타입 변경

##𝟣-𝟤. before_boxoffice 데이터 전처리.

In [33]:
before_boxoffice = before_boxoffice.drop('Unnamed: 0', axis = 1)
before_boxoffice

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,...,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,showRange,yearWeekTime,boxofficeType
0,1,1,0,OLD,20030410,실미도,2003-12-24,1019869500,45.0,651532000,...,2374524500,161201,103065,177.3,373849,50,882,20040102~20040104,200401,주말 박스오피스
1,2,2,0,OLD,20030316,반지의 제왕 : 왕의 귀환,2003-12-17,623744000,27.5,328630000,...,2673132000,100962,53390,112.2,430356,44,578,20040102~20040104,200401,주말 박스오피스
2,3,3,0,NEW,20030420,동해물과 백두산이,2003-12-31,331879000,14.6,331879000,...,564340000,52163,52163,100.0,88050,20,434,20040102~20040104,200401,주말 박스오피스
3,4,4,-1,OLD,20030404,러브 액츄얼리,2003-12-05,107668500,4.7,-6645500,...,1082576000,16369,-1428,-8.0,165883,11,158,20040102~20040104,200401,주말 박스오피스
4,5,5,0,NEW,20030426,더 캣,2003-12-31,75431000,3.3,75431000,...,108789000,12442,12442,100.0,17726,11,181,20040102~20040104,200401,주말 박스오피스
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8340,6,6,0,NEW,20192721,캣츠,2019-12-24,968403660,3.9,968403660,...,5838643210,110154,110154,100.0,696173,731,6504,20191227~20191229,201952,주말 박스오피스
8341,7,7,-2,OLD,20192401,포드 V 페라리,2019-12-04,498445920,2.0,-191995200,...,10349888380,53881,-21843,-28.8,1177035,181,874,20191227~20191229,201952,주말 박스오피스
8342,8,8,21,OLD,20198818,프린스 코기,2019-12-24,346360100,1.4,339548100,...,877404060,43730,42974,5684.4,112330,416,1606,20191227~20191229,201952,주말 박스오피스
8343,9,9,82,OLD,20199981,눈의 여왕4,2019-12-24,296118420,1.2,295554420,...,889321900,43501,43407,46177.7,119058,428,1484,20191227~20191229,201952,주말 박스오피스


In [34]:
datetime_nan = before_boxoffice[before_boxoffice['openDt'].isna()]
datetime_nan

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,...,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,showRange,yearWeekTime,boxofficeType


In [35]:
before_boxoffice['openDt'].isna().sum()

0

In [36]:
# openDt 데이터 타입 변경 : 날짜
filtered_df = before_boxoffice[~before_boxoffice['openDt'].str.contains(r'(-.*?){2,}')].reset_index(drop=True)
filtered_df


<ipython-input-36-cb1a4a6efe2a>:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df = before_boxoffice[~before_boxoffice['openDt'].str.contains(r'(-.*?){2,}')].reset_index(drop=True)


,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,...,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,showRange,yearWeekTime,boxofficeType
0,10,10,0,NEW,20040452,"[통합]회고전, 기획전 등",,8396300,0.3,8396300,...,61695300,1597,1597,100.0,11259,2,24,20040528~20040530,200422,주말 박스오피스
1,10,10,0,OLD,20040452,"[통합]회고전, 기획전 등",,13554500,0.2,5158200,...,87482300,2524,927,58.0,16139,2,30,20040604~20040606,200423,주말 박스오피스
2,9,9,3,OLD,20040452,"[통합]회고전, 기획전 등",,43221000,0.7,32824000,...,309098800,7433,4950,199.4,57640,6,77,20041029~20041031,200444,주말 박스오피스
3,8,8,6,OLD,20050001,2005 [통합]패키지영화,,52480500,0.6,32325500,...,200124500,7485,4656,164.6,24388,8,109,20050617~20050619,200524,주말 박스오피스
4,9,9,-1,OLD,20050001,2005 [통합]패키지영화,,50488500,0.6,-1992000,...,280015000,7097,-388,-5.2,36063,8,110,20050624~20050626,200525,주말 박스오피스
5,9,9,5,OLD,20050001,2005 [통합]패키지영화,,48313500,0.5,34889500,...,427138000,7188,5733,394.0,57749,15,132,20050715~20050717,200528,주말 박스오피스
6,9,9,0,OLD,20050001,2005 [통합]패키지영화,,48644000,0.5,330500,...,496499500,7128,-60,-0.8,68229,17,152,20050722~20050724,200529,주말 박스오피스
7,9,9,2,OLD,20050001,2005 [통합]패키지영화,,65123000,0.5,24800000,...,652319000,9311,3784,68.5,90931,17,146,20050805~20050807,200531,주말 박스오피스
8,9,9,6,OLD,20050001,2005 [통합]패키지영화,,45122000,0.4,14511000,...,834928000,6159,1760,40.0,116421,8,133,20050909~20050911,200536,주말 박스오피스
9,10,10,-1,OLD,20050001,2005 [통합]패키지영화,,51587500,0.4,6465500,...,904999500,7313,1154,18.7,126523,9,150,20050916~20050918,200537,주말 박스오피스


조회가 되지 않은 값들 확인하기.

In [37]:
filtered_df.loc[1,'openDt']

' '

In [38]:
filtered_df = before_boxoffice[before_boxoffice['openDt'] != ' ']
filtered_df.shape, before_boxoffice.shape

((8321, 21), (8345, 21))

2011년 이전 데이터 제거.

In [39]:
filtered_df['openDt'] = pd.to_datetime(filtered_df['openDt'])
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8321 entries, 0 to 8344
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   rnum           8321 non-null   int64         
 1   rank           8321 non-null   int64         
 2   rankInten      8321 non-null   int64         
 3   rankOldAndNew  8321 non-null   object        
 4   movieCd        8321 non-null   object        
 5   movieNm        8321 non-null   object        
 6   openDt         8321 non-null   datetime64[ns]
 7   salesAmt       8321 non-null   int64         
 8   salesShare     8321 non-null   float64       
 9   salesInten     8321 non-null   int64         
 10  salesChange    8321 non-null   float64       
 11  salesAcc       8321 non-null   int64         
 12  audiCnt        8321 non-null   int64         
 13  audiInten      8321 non-null   int64         
 14  audiChange     8321 non-null   float64       
 15  audiAcc        8321 non-nu

<ipython-input-39-36c386dc8695>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['openDt'] = pd.to_datetime(filtered_df['openDt'])


In [40]:
filtered_before_boxofffice = filtered_df[filtered_df['openDt']>= '2011-01-01']
filtered_before_boxofffice

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,...,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,showRange,yearWeekTime,boxofficeType
3656,2,2,12,OLD,20101399,심장이 뛴다,2011-01-05,2644812000,20.5,2561712000,...,3615981000,342548,331353,2959.8,481023,445,5106,20110107~20110109,201101,주말 박스오피스
3662,8,8,0,NEW,20102413,극장판 메탈 베이블레이드 VS 태양 작열의 침략자 솔블레이즈,2011-01-05,260444000,2.0,260444000,...,311313000,38316,38316,100.0,46127,111,807,20110107~20110109,201101,주말 박스오피스
3665,1,1,0,NEW,20102421,메가마인드,2011-01-13,3226755500,26.7,3226755500,...,3732477000,316638,316638,100.0,370620,414,4655,20110114~20110116,201102,주말 박스오피스
3668,4,4,0,NEW,20110031,러브 & 드럭스,2011-01-13,1457666000,12.1,1457666000,...,1713781000,182397,182397,100.0,216849,284,3311,20110114~20110116,201102,주말 박스오피스
3669,5,5,0,NEW,20110029,시즌 오브 더 위치: 마녀 호송단,2011-01-13,1399232500,11.6,1399232500,...,1627342000,182335,182335,100.0,213750,218,3460,20110114~20110116,201102,주말 박스오피스
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8340,6,6,0,NEW,20192721,캣츠,2019-12-24,968403660,3.9,968403660,...,5838643210,110154,110154,100.0,696173,731,6504,20191227~20191229,201952,주말 박스오피스
8341,7,7,-2,OLD,20192401,포드 V 페라리,2019-12-04,498445920,2.0,-191995200,...,10349888380,53881,-21843,-28.8,1177035,181,874,20191227~20191229,201952,주말 박스오피스
8342,8,8,21,OLD,20198818,프린스 코기,2019-12-24,346360100,1.4,339548100,...,877404060,43730,42974,5684.4,112330,416,1606,20191227~20191229,201952,주말 박스오피스
8343,9,9,82,OLD,20199981,눈의 여왕4,2019-12-24,296118420,1.2,295554420,...,889321900,43501,43407,46177.7,119058,428,1484,20191227~20191229,201952,주말 박스오피스


#### before_boxoffice 데이터 전처리 작업내용
* 2011년 이전 데이터 제외.
* OpenDt 타입 변경 -> datetime 및 Null값 제거.

## 𝟣-𝟥. after_boxoffice 데이터 전처리.

In [41]:
after_boxoffice = after_boxoffice.drop('Unnamed: 0', axis = 1)
after_boxoffice

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,...,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,showRange,yearWeekTime,boxofficeType
0,1,1,0,OLD,20192206,백두산,2019-12-19,4974627980,30.4,-5577379140,...,63308870470,565072,-635924,-52.9,7479389,1222,16822,20200103~20200105,202001,주말 박스오피스
1,2,2,0,OLD,20184571,천문: 하늘에 묻는다,2019-12-26,2973101060,18.1,-1785042810,...,13373622380,343199,-197742,-36.6,1611399,922,10556,20200103~20200105,202001,주말 박스오피스
2,3,3,38,OLD,20196272,미드웨이,2019-12-31,2718781120,16.6,2715661120,...,5818353580,306224,305912,98048.7,669797,842,8757,20200103~20200105,202001,주말 박스오피스
3,4,4,-1,OLD,20198414,시동,2019-12-18,2336951080,14.3,-1781487200,...,25727945170,264165,-198203,-42.9,3037781,825,8766,20200103~20200105,202001,주말 박스오피스
4,5,5,-1,OLD,20197803,겨울왕국 2,2019-11-21,990055340,6.0,-678594910,...,113542953120,112533,-78807,-41.2,13593927,644,4009,20200103~20200105,202001,주말 박스오피스
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2635,6,6,-3,OLD,20248082,동화지만 청불입니다,2025-01-08,272488040,5.0,-345796760,...,1465916720,27665,-37901,-57.8,157616,594,3435,20250117~20250119,202503,주말 박스오피스
2636,7,7,0,OLD,20245977,극장판 짱구는 못말려: 우리들의 공룡일기,2024-12-18,242954440,4.5,-64394580,...,6987330060,25716,-6744,-20.8,742520,499,1988,20250117~20250119,202503,주말 박스오피스
2637,8,8,1,OLD,20247693,위키드,2024-11-20,188082350,3.5,-117522170,...,21677201940,18800,-8116,-30.2,2195165,335,1346,20250117~20250119,202503,주말 박스오피스
2638,9,9,-1,OLD,20240193,무파사: 라이온 킹,2024-12-18,163148560,3.0,-132116480,...,8188072950,17247,-14120,-45.0,833832,415,1430,20250117~20250119,202503,주말 박스오피스


In [42]:
after_boxoffice['openDt']

,openDt
0,2019-12-19
1,2019-12-26
2,2019-12-31
3,2019-12-18
4,2019-11-21
...,...
2635,2025-01-08
2636,2024-12-18
2637,2024-11-20
2638,2024-12-18


In [43]:
datetime_nan = after_boxoffice[after_boxoffice['openDt'].isna()]
datetime_nan

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,...,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,showRange,yearWeekTime,boxofficeType


In [44]:
# openDt 데이터 타입 변경 : 날짜
filtered_df = after_boxoffice[~after_boxoffice['openDt'].str.contains(r'(-.*?){2,}')].reset_index(drop=True)
filtered_df

<ipython-input-44-52debafa14c0>:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df = after_boxoffice[~after_boxoffice['openDt'].str.contains(r'(-.*?){2,}')].reset_index(drop=True)


,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,...,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,showRange,yearWeekTime,boxofficeType
0,8,8,0,NEW,20201421,BIFAN2020 판타스틱 단편 걸작선 1,,46200000,1.5,46200000,...,52800000,7700,7700,100.0,8800,6,70,20200710~20200712,202028,주말 박스오피스
1,7,7,0,NEW,20218827,BIFAN2021 판타스틱 단편 걸작선 1,,58405000,0.5,58405000,...,67285000,11681,11681,100.0,13457,6,109,20210709~20210711,202127,주말 박스오피스
2,8,8,0,NEW,20210853,BIAF2021 한국 단편 경쟁 B,,18410000,0.2,18410000,...,20410000,3682,3682,100.0,4082,3,24,20211022~20211024,202142,주말 박스오피스
3,8,8,0,NEW,20225994,BIFAN2022 부천 초이스: 단편 1,,20700000,0.1,20700000,...,28300000,4140,4140,100.0,5660,1,23,20220708~20220710,202227,주말 박스오피스
4,9,9,0,NEW,20225979,BIFAN2022 부천 초이스: 단편 2,,18900000,0.1,18900000,...,23650000,3780,3780,100.0,4730,1,21,20220708~20220710,202227,주말 박스오피스
5,9,9,0,NEW,20240641,밤낚시,,17375000,0.1,17375000,...,20343000,16212,16212,100.0,16636,34,192,20240614~20240616,202424,주말 박스오피스
6,6,6,3,OLD,20240641,밤낚시,,10793000,0.1,-6582000,...,33084500,10146,-6066,-37.4,28731,33,179,20240621~20240623,202425,주말 박스오피스
7,8,8,-2,OLD,20240641,밤낚시,,7246500,0.0,-3546500,...,44484000,7247,-2899,-28.6,40131,28,127,20240628~20240630,202426,주말 박스오피스


In [45]:
filtered_df.loc[1,'openDt']

' '

In [46]:
filtered_after_boxoffice = after_boxoffice[after_boxoffice['openDt'] != ' ']
filtered_after_boxoffice.shape, after_boxoffice.shape

((2632, 21), (2640, 21))

중복행 찾기.

#### after_boxoffice 데이터 전처리 작업내용
* 2011년 이전 데이터 제외.
* OpenDt 타입 변경 -> datetime 및 Null값 제거.

### movieCd 일치 여부 확인

Kobis에서 가져온 데이터와 박스오피스에서 가져온 데이터가 일치하는지 확인해야함.
* 매출 부분으로 접근.

In [47]:
df1 = filtered_after_boxoffice[['movieCd', 'movieNm','salesAcc']]
df2 = filtered_kobis[['영화명', '영화코드','전국_매출액']]

df1.shape, df2.shape

((2632, 3), (19897, 3))

In [48]:
df1_filtered = df1.loc[df1.groupby(['movieCd','movieNm'])['salesAcc'].idxmax()]
df1_filtered

,movieCd,movieNm,salesAcc
1058,19838068,해탄적일천,20352940
167,19888010,시네마 천국,268935700
545,19910182,늑대와 춤을,51677820
2288,19950084,쇼생크 탈출,414947067
2256,19950095,레옹,488843702
...,...,...,...
2509,20249434,조커: 폴리 아 되,6323565000
2356,20249507,극장판 하이큐!! 쓰레기장의 결전,7323368219
2457,20249554,스픽 노 이블,264264710
2497,20249733,사랑의 하츄핑,10882238222


`트랜스포머 ONE` 을 기준으로 제대로 분류되었는지 확인합니다.

In [49]:
#트랜스포머 ONE
df1[df1['movieNm'] == '트랜스포머 ONE']

,movieCd,movieNm,salesAcc
2467,20249899,트랜스포머 ONE,360230770
2471,20249899,트랜스포머 ONE,1884542785
2484,20249899,트랜스포머 ONE,2914640205
2496,20249899,트랜스포머 ONE,3266219645


제대로 분류됨을 확인했습니다, kobis 또한 적용합니다.

In [60]:
df2['영화명'] = df2['영화명'].astype(str)
df2['영화코드'] = df2['영화코드'].astype(str)
df2['전국_매출액'] = df2['전국_매출액'].astype(int)
df2_filtered = df2.loc[df2.groupby(['영화코드', '영화명'])['전국_매출액'].idxmax()]
df2_filtered

,영화명,영화코드,전국_매출액
628,만추 리마스터링,19820019,6300514500
4517,해탄적일천,19838068,45827240
113,라이온 킹,19940256,41518977430
2634,큐어,19978805,250535969
3928,올드보이,20030350,56281683
...,...,...,...
19724,힘좋은 시아버지와 베란다,e,6000
4110,힙노시스: LP 커버의 전설,e,61213647
3790,힙노틱,e,78114011
19725,힙스터: 안개의 덫,e,5000


In [61]:
merge_df = pd.merge(df1_filtered, df2_filtered, left_on='movieCd', right_on='영화코드', how='inner')
merge_df

,movieCd,movieNm,salesAcc,영화명,영화코드,전국_매출액
0,19838068,해탄적일천,20352940,해탄적일천,19838068,45827240
1,19978805,큐어,122001600,큐어,19978805,250535969
2,20078386,블라인드,138729200,블라인드,20078386,18361430500
3,20080269,해피 투게더,140940120,해피 투게더,20080269,204452450
4,20112561,원 데이,1658242700,원 데이,20112561,1492933500
...,...,...,...,...,...,...
888,20249434,조커: 폴리 아 되,6323565000,조커: 폴리 아 되,20249434,6374984320
889,20249507,극장판 하이큐!! 쓰레기장의 결전,7323368219,극장판 하이큐!! 쓰레기장의 결전,20249507,7634168967
890,20249554,스픽 노 이블,264264710,스픽 노 이블,20249554,436493110
891,20249733,사랑의 하츄핑,10882238222,사랑의 하츄핑,20249733,10977685042


In [64]:
merge_df[merge_df['movieCd'] != merge_df['영화코드']]

,movieCd,movieNm,salesAcc,영화명,영화코드,전국_매출액


In [65]:
! mkdir -p /content/-Screen-Wizard/Data/processed

추가적으로 각각의 다른 df를 담기 위한 폴더 생성.

In [66]:
! mkdir /content/-Screen-Wizard/Data/processed/Kobis_data
! mkdir /content/-Screen-Wizard/Data/processed/before
! mkdir /content/-Screen-Wizard/Data/processed/after

In [68]:
# 현재 생성된 파일입니다. 생성하지마세요!
filtered_kobis.to_csv('/content/-Screen-Wizard/Data/processed/Kobis_data/filtered_kobis.csv', index = False)
filtered_before_boxofffice.to_csv('/content/-Screen-Wizard/Data/processed/before/filtered_before_boxoffice.csv', index = False)
filtered_after_boxoffice.to_csv('/content/-Screen-Wizard/Data/processed/after/filtered_after_boxoffice.csv', index = False)